In [1]:
#imports
import numpy as np
import pandas as pd
import requests
import json
import os
import re

In [2]:
df = pd.read_csv('Input/IMDb_movies.csv',encoding = 'latin-1')
'''
#visual
print(df.dtypes)
print(df.isnull().sum())
display(df.count())'''

'\n#visual\nprint(df.dtypes)\nprint(df.isnull().sum())\ndisplay(df.count())'

In [3]:
df = df.drop(columns = ['reviews_from_users','reviews_from_critics','votes','original_title','date_published','usa_gross_income','writer'])

In [4]:
from Cleaning import removeNaN
from Cleaning import decader
from Cleaning import dollar_to_int


In [5]:
df = removeNaN(df, df['worlwide_gross_income'])
df = removeNaN(df, df['metascore'])
df = removeNaN(df, df['budget'])

In [6]:
#visual
'''print(df.isnull().sum())'''

'print(df.isnull().sum())'

In [7]:
df =df[df['budget'].str.startswith(('$'))].reset_index(drop=True)
df['budget']= dollar_to_int(df['budget'])
df['worlwide_gross_income']= dollar_to_int(df['worlwide_gross_income'])


In [8]:
df['decade']= decader(df['year'])
df['profit']= df['worlwide_gross_income'] - df['budget']
df['profit_ratio'] =round(df['profit']/df['budget'], 2)

In [9]:
df = df.sort_values(by=['profit_ratio'],ascending=False).reset_index(drop=True)

In [10]:
df.iloc[0, df.columns.get_loc('budget')] = 10700000


In [11]:
df['decade']= decader(df['year'])
df['profit']= df['worlwide_gross_income'] - df['budget']
df['profit_ratio'] =round(df['profit']/df['budget'], 2)
df['score']= decader(df['metascore']).astype('int64')
df = df.sort_values(by=['profit_ratio'],ascending=False).reset_index(drop=True)

In [12]:
df = df.sort_values(by=['worlwide_gross_income'],ascending=False).reset_index(drop=True)

In [13]:
decade_table = round((df.groupby('decade').agg({'budget':['max','mean'],"worlwide_gross_income":['max','mean'],"profit":["max",'mean']}))/1000000 , 0).astype('int64')
decade_table['profit_ratio']= round(df.groupby('decade').agg({"profit_ratio":"mean"}),2)
decade_table['n_movies']=df.groupby('decade').agg({'title':'count'})
decade_table

budget      worlwide_gross_income      profit      profit_ratio  \
          max mean                   max mean    max mean                
decade                                                                   
1920        1    1                     0    0     -1   -1        -0.97   
1930        4    2                   402   68    398   67        25.04   
1940        3    2                   122   19    119   18         7.33   
1950       15    3                    95   17     92   15         4.75   
1960       44    5                   216   28    212   23         6.84   
1970       55    7                   776   56    765   50        13.28   
1980       70   14                   793   49    782   35         3.81   
1990      200   31                  2187   85   1987   54         7.16   
2000      300   33                  2790   80   2553   47         7.57   
2010      356   39                  2798  123   2442   85         2.40   

       n_movies  
                 
decade           
1920          1  
1930          9  
1940         12  
1950         34  
1960         72  
1970        163  
1980        577  
1990        931  
2000       2265  
2010       2128

In [14]:
score_table = round((df.groupby('score').agg({'budget':['max','mean'],"worlwide_gross_income":['max','mean'],"profit":["max",'mean']}))/1000000 , 0).astype('int64')
score_table['profit_ratio']= round(df.groupby('score').agg({"profit_ratio":"mean"}),2)
score_table['n_movies']=df.groupby('score').agg({'title':'count'})
score_table

budget      worlwide_gross_income      profit      profit_ratio n_movies
         max mean                   max mean    max mean                      
score                                                                         
0         73   13                    85   10     55   -3        -0.29       23
10       109   22                   247   27    168    5         1.51      151
20       217   26                   605   41    388   15         1.08      394
30       230   33                  1104   62    894   30         1.73      865
40       300   35                  1124   83    955   48         1.55     1151
50       300   35                  1670  103   1520   67         2.17     1188
60       321   32                  2048  103   1727   71        14.17     1131
70       356   30                  2798  111   2442   81         4.42      745
80       317   26                  2790  136   2553  109        17.53      408
90       200   21                  1120  142   1026  121         9.54      124
100       15    4                   246   35    240   31         8.42       12